In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import allel
import statistics as stat
import statsmodels.stats.multitest as stats
import math

In [ ]:
# run the perl script "fst-sliding.pl" of the suite popoolation2 using the command:
#./fst-sliding.pl --input sync_Dlabrax.txt --output sync_Dlabrax.fst --min-count 1 
# --min-coverage 1 --max-coverage 100000 --window-size=1 --step-size=1 
# --pool-size=12:23:25:25:25:25:25:25:25:25:25:25:25:25:25:25:25:25:25:25:25:11:12:25
# window-size=1 and step-size=1 are to calculate the fst for every SNP
sync_Dlabrax_fst = pd.read_csv(r"C:\Users\arist\Downloads\sync_Dlabrax_LG10_popoolation.fst", sep="\t", header=None)
sync_Dlabrax_fst

In [ ]:
# focus on fst differences between the farmed and the wild populations
# pops1-10 are farmed; pops10-20 are wild
# select cols 16-23, 38-45, 59-66, 79-86, 98-105, 116-123, 133-140, 149-156, 164-171, 
# 178-185, 191-198, 203-210
indices = [16, 17, 18, 19, 20, 21, 22, 23,
           34, 35, 36, 37, 38, 39, 40, 41,
           51, 52, 53, 54, 55, 56, 57, 58,
           67, 68, 69, 70, 71, 72, 73, 74,
           82, 83, 84, 85, 86, 87, 88, 89,
           96, 97, 98, 99, 100, 101, 102, 103,
           109, 110, 111, 112, 113, 114, 115, 116,
           121, 122,123, 124, 125, 126, 127, 128,
           132, 133, 134, 135, 136, 137, 138 ,139,
           142, 143, 144, 145, 146, 147, 148, 149,
           151, 152, 153, 154, 155, 156, 157, 158,
           159, 160, 161, 162, 163, 164, 165, 166]


sync_Dlabrax_fst_filtered = sync_Dlabrax_fst.iloc[:, indices]
sync_Dlabrax_fst_filtered

In [ ]:
avg_FST_perSNP_Dlabrax = []

for row in range(len(sync_Dlabrax_fst_filtered)):
    if row%10000 == 0:
        print(row, end=" ")
    avg_FST_perSNP_perComparison = []
    for element in range(len(list(sync_Dlabrax_fst_filtered.iloc[row,:]))):
        num_ = float(list(sync_Dlabrax_fst_filtered.iloc[row,:])[element].split("=")[1])
        avg_FST_perSNP_perComparison.append(num_)
    avg_FST_perSNP_Dlabrax.append(stat.mean(avg_FST_perSNP_perComparison))

In [ ]:
# calculate the rolling average of Fst over 500 SNPs
sync_Dlabrax_fst_df = pd.DataFrame(avg_FST_perSNP_Dlabrax, columns=["Fst"])
sync_Dlabrax_fst_df['index1'] = sync_Dlabrax_fst_df.index
sync_Dlabrax_fst_df["500rollingAVG"] = sync_Dlabrax_fst_df["Fst"].rolling(window=500).mean()
sync_Dlabrax_fst_df

In [ ]:
pos = sync_Dlabrax_fst[1]

sync_Dlabrax_fst_df['pos'] = pos
sync_Dlabrax_fst_df

In [ ]:
# plot the rolling average Fst
sns.set_style("darkgrid")
plt.figure(figsize=(16, 8))
plt.plot(sync_Dlabrax_fst_df["500rollingAVG"], linestyle='dashed', color = 'tab:green')
# plt.axvline(x = 150396, color = 'r')
# plt.axvline(x = 150397, color = 'r')
plt.title("Fst rolling average over 5000 SNPs")
plt.xlabel('#bin') 
plt.ylabel('Fst rolling average')
plt.show()
# INTERESTING FINDING: which GENES are around those two peaks????

In [ ]:
sync_Dlabrax_fst_df.to_csv(r"C:\Users\arist\Downloads\LG10_Dlabrax_popoolation.csv", header=True, index=False, sep=" ")

In [ ]:
# test whether any differences in allele frequencies are statistically significant
# using the script "fisher-test.pl" of popoolation
# USAGE for Dlabrax: ./fisher-test.pl --input sync_Dlabrax.txt --output sync_Dlabrax.fet 
# --min-count 1 --min-coverage 1 --max-coverage 10000
sync_Dlabrax_fet = pd.read_csv("./popoolation2_1201/sync_Dlabrax.fet", sep="\t", header=None, low_memory=False)

In [ ]:
# focus on fst differences between the farmed and the wild populations
# pops1-12 are farmed; pops13-20 are wild
# select cols select cols 16-23, 38-45, 59-66, 79-86, 98-105, 116-123, 133-140, 149-156, 164-171, 
# 178-185, 191-198, 203-210
indices = [16, 17, 18, 19, 20, 21, 22, 23,
           38, 39, 40, 41, 42, 43, 44, 45,
           59, 60, 61, 62, 63, 64, 65, 66,
           79, 80, 81, 82, 83, 84, 85, 86,
           98, 99, 100, 101, 102, 103, 104, 105,
           116, 117, 118, 119, 120, 121, 122, 123,
           133, 134, 135, 136, 137, 138, 139, 140,
           149, 150, 151, 152, 153, 154, 155, 156,
           164, 165, 166, 167, 168, 169, 170, 171,
           178, 179, 180, 181, 182, 183, 184, 185, 
           191, 192, 193, 194, 195, 196, 197, 198,
           203, 204, 205, 206, 207, 208, 209, 210]
sync_Dlabrax_fet_filtered = sync_Dlabrax_fet.iloc[:, indices]

In [ ]:
# if a population is below "--min-count 1" and/or "--min-coverage 1" at the perl script above
# then "na" is returned. To accomodate these few cases, 
# we drop those cases from the filtered file
# and use "pass" to not include them to the generated list
# IMPORTANT: SIGNIFICANCE IS REPORTED AS -LOG(P-VALUE)!!!!
# -LOG(0.01) = 2 (thus we search for values > 2)
# The loop below calculates the PERCENTAGE of pairwise comparisons per SNP that have -LOG(P-val) > 2 (p < 1%)

PERCENT_SIGN_FET_perSNP_Dlabrax = []
dropped_indices = []

for row in range(len(sync_Dlabrax_fet_filtered)):
    if row%10000 == 0:
        print(row, end=" ")
    count_sign_FET = 0
    try:
        for element in range(len(list(sync_Dlabrax_fet_filtered.iloc[row,:]))):
            num_ = round(float(sync_Dlabrax_fet_filtered.iloc[row,element].split("=")[1]), 3)
            if num_ > 1.3:  # 0.05 significance
                count_sign_FET += 1
        PERCENT_SIGN_FET_perSNP_Dlabrax.append(round(count_sign_FET/142, 3)*100)  # 142=tot pairwise comps
    except ValueError:
        dropped_indices.append(row)
        pass

# =======================================================================================================
# another variation of the above "for" loop, which estimates the AVERAGE -LOG(P-val) per row

# AVERAGE_FET_per_SNP_Dlabrax = []
# dropped_indices = []

# for row in range(len(sync_Dlabrax_fet_filtered)):
#     FET_values_per_SNP = []
#     if row%10000 == 0:
#         print(row, end=" ")
#     try:
#         for element in range(len(list(sync_Dlabrax_fet_filtered.iloc[row,:]))):
#             num_ = round(float(sync_Dlabrax_fet_filtered.iloc[row,element].split("=")[1]), 3)
#             FET_values_per_SNP.append(num_)
#         row_mean = stat.mean(FET_values_per_SNP)
#         AVERAGE_FET_per_SNP_Dlabrax.append(row_mean)
#     except ValueError:
#         dropped_indices.append(row)
#         pass
# =======================================================================================================

In [ ]:
# 81 cases with "na" that will be dropped
len(dropped_indices)
sync_Dlabrax_fet_filtered = sync_Dlabrax_fet_filtered.drop(index = dropped_indices)

# make certain to round the resulted averages to 3 decimals
PERCENT_SIGN_FET_perSNP_Dlabrax = [round(x, 3) for x in PERCENT_SIGN_FET_perSNP_Dlabrax]

# add averages as new column to the "sync_Dlabrax_fet_filtered" df
sync_Dlabrax_fet_filtered["PERCENT_SIGN"] = PERCENT_SIGN_FET_perSNP_Dlabrax

In [ ]:
# =======================================================================================================
# same as just above, but for the variation with the AVERAGE -LOG(P-val) per row
# 81 cases with "na" that will be dropped
len(dropped_indices)
sync_Dlabrax_fet_filtered = sync_Dlabrax_fet_filtered.drop(index = dropped_indices)

# make certain to round the resulted averages to 3 decimals
AVERAGE_FET_per_SNP_Dlabrax = [round(x, 3) for x in AVERAGE_FET_per_SNP_Dlabrax]

# add averages as new column to the "sync_Dlabrax_fet_filtered" df
sync_Dlabrax_fet_filtered["AVERAGE_SIGN"] = AVERAGE_FET_per_SNP_Dlabrax

In [ ]:
# calculate the rolling average
sync_Dlabrax_fet_filtered["5000rollingAVG-PERCENT"] = sync_Dlabrax_fet_filtered["PERCENT_SIGN"].rolling(window=5000).mean()

# plot the rolling average Fst
sns.set_style("darkgrid")
plt.figure(figsize=(16, 8))
plt.plot(sync_Dlabrax_fet_filtered["5000rollingAVG-PERCENT"], linestyle='dashed', color = 'tab:green')
plt.axvline(x = 150396, color = 'r')
plt.axvline(x = 150397, color = 'r')
plt.title("Percent of pairwise comparisons with -LOG(P-val) > 2 (p < 1%)")
plt.xlabel('#bin') 
plt.ylabel('Percent')
plt.show()

In [ ]:
# CORRECT FOR MULTIPLE TESTING WITH THE BH METHOD

# the "for" loop of above, which estimates the AVERAGE -LOG(P-val) per row
# modified to correct for multiple test

AVERAGE_FET_per_SNP_Dlabrax_BHcor = []
dropped_indices = []

for row in range(len(sync_Dlabrax_fet_filtered)):
    FET_values_per_SNP = []
    if row%10000 == 0:
        print(row, end=" ")
    try:
        for element in range(len(list(sync_Dlabrax_fet_filtered.iloc[row,:]))):
            num_ = round(float(sync_Dlabrax_fet_filtered.iloc[row,element].split("=")[1]), 3)
            num_ = 10**(-num_)  # convert num_ from log10(p-value) to p-value
            FET_values_per_SNP.append(num_)
        # run the multiple correction
        FET_values_per_SNP_cor = stats.multipletests(FET_values_per_SNP, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
        # convert the corrected values back to log10
        FET_values_per_SNP_cor = [-math.log10(x) for x in list(FET_values_per_SNP_cor)]
        row_mean_cor = stat.mean(FET_values_per_SNP_cor)
        AVERAGE_FET_per_SNP_Dlabrax_BHcor.append(row_mean_cor)
    except ValueError:
        dropped_indices.append(row)
        pass

In [ ]:
# =======================================================================================================
# same as just above, but for the variation with the AVERAGE -LOG(P-val) per row
# 81 cases with "na" that will be dropped
len(dropped_indices)
sync_Dlabrax_fet_filtered = sync_Dlabrax_fet_filtered.drop(index = dropped_indices)

# make certain to round the resulted averages to 3 decimals
AVERAGE_FET_per_SNP_Dlabrax_BHcor = [round(x, 3) for x in AVERAGE_FET_per_SNP_Dlabrax_BHcor]

# add averages as new column to the "sync_Dlabrax_fet_filtered" df
sync_Dlabrax_fet_filtered["AVERAGE_SIGN_BHcor"] = AVERAGE_FET_per_SNP_Dlabrax_BHcor
# =======================================================================================================

In [ ]:
# same as above, but for the AVERAGE -LOG(P-val) per row
# calculate the rolling average
sync_Dlabrax_fet_filtered["500rollingAVG_BHcor"] = sync_Dlabrax_fet_filtered["AVERAGE_SIGN_BHcor"].rolling(window=500).mean()

# plot the rolling average Fst
sns.set_style("darkgrid")
plt.figure(figsize=(16, 8))
plt.plot(sync_Dlabrax_fet_filtered["5000rollingAVG_BHcor"], linestyle='dashed', color = 'tab:green')
plt.axvline(x = 150396, color = 'r')
plt.axvline(x = 150397, color = 'r')
plt.axhline(y = 1.3, color = 'r')
plt.title("Rolling averaged significance as -LOG(P-val)")
plt.xlabel('#bin') 
plt.ylabel('-LOG(P-val)_BHcor')
plt.show()
# Maybe too conservative that rolling average of 5000?
# What I would do is focus on individual points that stayed significant - see cell below

In [ ]:
# focus on individual points with fdr-corrected data
# notice that index "100426" (first peak) has -log(p-val) = 5.834 ==> pval = 1.47e-06
# also index "165381" (second peak) has -log(p-val) = 5.407 ==> pval = 3.91e-06
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
sync_Dlabrax_fet_filtered[sync_Dlabrax_fet_filtered["AVERAGE_SIGN_BHcor"] > 1.3].sort_values(by=['AVERAGE_SIGN_BHcor'], ascending=False).head(500)